In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
import re

# <u>Useful pandas settings </u>


In [2]:
# disp all columns 
pd.set_option('display.max_columns', None)
# disp n rows 
pd.set_option('display.max_columns', 5)



# <u>Basic DataFrame Manipulation </u>


 ### Filter based on column value

In [3]:
accomodation = pd.read_csv("accomodation.csv")


In [4]:
accomodation.head(2)

,name,city,country_code,size
0,Pensiunea Sovirag,Sic,ro,16
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42


In [5]:

france = accomodation.loc[accomodation["country_code"] == "fr",['name','size']]
france.head()

,name,size
12,Le PrĂ© du Lac,60
24,La Maison d'Hoursentut,13
46,HĂ´tel-Restaurant de La Paix,27
57,Premiere Classe Les Ulis,64
106,Eterlous,16


 ### Apply a transformation to all cells of data set

In [6]:
france = france.applymap(lambda cell: len(str(cell)))
france.head()

,name,size
12,14,2
24,22,2
46,28,2
57,24,2
106,8,2


 ### Return a value on a series based on another series

In [7]:
accomodation = pd.read_csv("accomodation.csv")


In [8]:
accomodation.head()

,name,city,country_code,size
0,Pensiunea Sovirag,Sic,ro,16
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42
2,Duvan Hotell,Uppsala,se,32
3,Wessex Hotel,Street Nr Glastonbury,gb,50
4,Alexandria Hotel,Thessaloniki,gr,28


In [9]:
# method 1 -> using loc --> easy if just need one value returned
accomodation["range"]=""
accomodation.loc[(accomodation["size"] > 12) & (accomodation["size"] < 30), "range"] = "Small"


In [10]:
accomodation

,name,city,country_code,size,range
0,Pensiunea Sovirag,Sic,ro,16,Small
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42,
2,Duvan Hotell,Uppsala,se,32,
3,Wessex Hotel,Street Nr Glastonbury,gb,50,
4,Alexandria Hotel,Thessaloniki,gr,28,Small
...,...,...,...,...,...
495,Casa Grande Boutique Hotel,Ciudad Bolivar,ve,15,Small
496,CabaĂ±as Arepo - Barrancas de Lolita,Areponapuchi,mx,12,
497,Lanting Inn,Dunhuang,cn,13,Small
498,Hotel Belveder,Pag,hr,26,Small


In [11]:
# method 2 -> using function --> better way. 
# you don't have to define the function per say, you could include it into the apply
def calculate_size(size): 
    if size > 150: 
        x = "very big" 
    elif size < 150 and size >= 100: 
        x = "big"
    elif size < 100 and size >= 30:
        x="medium"
    else: 
        x = "Small" 
    return x

accomodation["range2"] = accomodation.apply(lambda row: calculate_size(row["size"]), axis=1)


In [12]:
accomodation

,name,city,...,range,range2
0,Pensiunea Sovirag,Sic,...,Small,Small
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,...,,medium
2,Duvan Hotell,Uppsala,...,,medium
3,Wessex Hotel,Street Nr Glastonbury,...,,medium
4,Alexandria Hotel,Thessaloniki,...,Small,Small
...,...,...,...,...,...
495,Casa Grande Boutique Hotel,Ciudad Bolivar,...,Small,Small
496,CabaĂ±as Arepo - Barrancas de Lolita,Areponapuchi,...,,Small
497,Lanting Inn,Dunhuang,...,Small,Small
498,Hotel Belveder,Pag,...,Small,Small


 ### Map a dataframe with a dimension table

In [13]:
accomodation = pd.read_csv("accomodation.csv")
country_code_dimension = pd.read_csv("country_code.csv")

In [14]:
accomodation.head()

,name,city,country_code,size
0,Pensiunea Sovirag,Sic,ro,16
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42
2,Duvan Hotell,Uppsala,se,32
3,Wessex Hotel,Street Nr Glastonbury,gb,50
4,Alexandria Hotel,Thessaloniki,gr,28


In [15]:
country_code_dimension.head()

,Name,Code
0,Afghanistan,AF
1,Aland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [16]:

# column that needs to be mapped as index
country_code_dimension.set_index('Code', inplace= True)
#transform df into dictionary 
country_code_dimension = country_code_dimension.to_dict()

In [17]:
accomodation['country'] = accomodation["country_code"].str.upper().map(country_code_dimension['Name']) 
#use map function to map dimension table with ["country_code"] column of accomodation dataframe. 
#Don't forget to capitalize, you need a perfect string match

In [18]:
accomodation

,name,city,country_code,size,country
0,Pensiunea Sovirag,Sic,ro,16,Romania
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42,Germany
2,Duvan Hotell,Uppsala,se,32,Sweden
3,Wessex Hotel,Street Nr Glastonbury,gb,50,United Kingdom
4,Alexandria Hotel,Thessaloniki,gr,28,Greece
...,...,...,...,...,...
495,Casa Grande Boutique Hotel,Ciudad Bolivar,ve,15,"Venezuela, Bolivarian Republic of"
496,CabaĂ±as Arepo - Barrancas de Lolita,Areponapuchi,mx,12,Mexico
497,Lanting Inn,Dunhuang,cn,13,China
498,Hotel Belveder,Pag,hr,26,Croatia


# <u> Pandas and DateTime</u>


 ### Convert series to datetime format

In [19]:
import datetime as dt

In [20]:
data = [["13/02/1986", "Jim was born"], ["16-07-09" ,"Jim got married"], ["19/10/2020","Jim had a child"], [ "09-12-2054", "Jim died"] ]


jim = pd.DataFrame(data, columns = ['Date', 'Lifeof Jim'])
jim

,Date,Lifeof Jim
0,13/02/1986,Jim was born
1,16-07-09,Jim got married
2,19/10/2020,Jim had a child
3,09-12-2054,Jim died


In [21]:
jim["Date"] = pd.to_datetime(jim['Date'])
jim

,Date,Lifeof Jim
0,1986-02-13,Jim was born
1,2009-07-16,Jim got married
2,2020-10-19,Jim had a child
3,2054-09-12,Jim died


 ### Extract a specific unit from datetime


In [22]:
jim['Year'] = jim['Date'].dt.year # returns as an int
jim["Month"] = jim["Date"].dt.strftime("%B") # returns as a string
print(jim.head())
print(jim.dtypes)


        Date       Lifeof Jim  Year      Month
0 1986-02-13     Jim was born  1986   February
1 2009-07-16  Jim got married  2009       July
2 2020-10-19  Jim had a child  2020    October
3 2054-09-12         Jim died  2054  September
Date          datetime64[ns]
Lifeof Jim            object
Year                   int64
Month                 object
dtype: object


# <u> Manipulating Strings </u>


 ### Replacing strings in a series from a dictionary

In [23]:
mispelled_data = pd.read_csv("mispelled_data.csv") ## dataframe with lots of mispelled data (usually repeated mispells)
string_correction = pd.read_csv("string_correction.csv") ## dataframe wih corrections

In [24]:
mispelled_data.head()

,mispelled_string
0,candi
1,helo my name is Jeff
2,we are going to San Francesco tomorrow
3,I like to eate piza
4,I need to read more bookes to spel better


In [25]:
string_correction.head()

,wrong_string,right_string
0,candi,candy
1,helo,hello
2,San Francesco,San Francisco
3,eate,eat
4,piza,pizza


In [26]:
string_correction = string_correction.set_index("wrong_string") # set wrong string as index
string_correction = string_correction.to_dict() #tranform df into dic

In [27]:
def find_replace(string): #define functions that replaces wrong string by right string
    for item in string_correction['right_string'].keys():
        # sub item for item's paired value in dictionary
        string = re.sub(item, string_correction['right_string'][item], string)
    return string

In [29]:
mispelled_data['rightly_spelled_string'] =\
                     mispelled_data.apply(lambda row:\
                     find_replace(str(row["mispelled_string"])),\
                     axis=1) # apply function to mispelled string series

In [30]:
mispelled_data ## strings are replaced! 


,mispelled_string,rightly_spelled_string
0,candi,candy
1,helo my name is Jeff,hello my name is Jeff
2,we are going to San Francesco tomorrow,we are going to San Francisco tomorrow
3,I like to eate piza,I like to eat pizza
4,I need to read more bookes to spel better,I need to read more books to spel better


 ### "If contains" algorithm

In [31]:
# Here we are trying to format a name that can be written in multiple ways.
# We have a dictionary that tells us which strings can correspond to a specific brand name
# we need to write a function that returns the brand name in a new column if the original name contains a specific string
# several strings can return the same brand name


data = [["mcdonalds central london", ""], ["pizza hut downtown new york" ,""], ["dominos new jersey",""], [ "mac donald berlin", ""] ]

restaurants = pd.DataFrame(data, columns = ['restaurant_name', 'brand_name'])
    
brand_dictionary = {
"mcdonalds" : "McDonald's",
"mac donald" : "McDonald's",
"dominos" : "Dominos Pizza",
"pizza hut" : "Pizza Hut"}

def get_name(restaurant, dct):
    for r in dct:
        if r in restaurant:
            return dct[r]
    return '-'

restaurants['brand_name'] = restaurants['restaurant_name'].apply(lambda x: get_name(x, brand_dictionary))
restaurants

,restaurant_name,brand_name
0,mcdonalds central london,McDonald's
1,pizza hut downtown new york,Pizza Hut
2,dominos new jersey,Dominos Pizza
3,mac donald berlin,McDonald's


### Word Counter

In [32]:
from collections import Counter
accomodation = pd.read_csv("accomodation.csv")
accomodation.head()

,name,city,country_code,size
0,Pensiunea Sovirag,Sic,ro,16
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,de,42
2,Duvan Hotell,Uppsala,se,32
3,Wessex Hotel,Street Nr Glastonbury,gb,50
4,Alexandria Hotel,Thessaloniki,gr,28


In [33]:
#use counter to store number of times a string is returned in an object
word_count = Counter(' '.join(accomodation['name']).split(' ')).items() 

In [34]:
word_count

dict_items([('Pensiunea', 1), ('Sovirag', 1), ('Bio-Vitalhotel', 1), ('Falkenhof', 1), ('Duvan', 1), ('Hotell', 2), ('Wessex', 1), ('Hotel', 205), ('Alexandria', 1), ('HotelovĂ˝', 1), ('resort', 1), ('Ĺ\xa0ikland', 1), ('Markets', 1), ('Residence', 4), ('Inn', 51), ('Houston', 1), ('Medical', 1), ('Center', 4), ('/', 3), ('NRG', 1), ('Park', 6), ('Sea', 1), ('View', 5), ('Heights', 1), ('Villa', 7), ('Montego', 1), ('Bay', 3), ('YangShuo', 1), ('Eden', 1), ('YHA', 1), ('Casablanca', 1), ('Palace', 4), ('Sagar', 1), ('Le', 7), ('PrĂ©', 1), ('du', 3), ('Lac', 1), ('Dionyssos', 1), ("Rantun's", 1), ('Place', 1), ('The', 21), ('Gateway', 3), ('M', 1), ('G', 1), ('Road', 3), ('NH', 1), ('Collection', 1), ('Victoria', 3), ('Raoum', 1), ("Innkeeper's", 1), ('Lodge', 8), ('Doncaster,', 1), ('Bessacarr', 1), ('Pivnice', 1), ('ve', 1), ('Dvorci', 1), ('-', 11), ('JĂ\xadtrava', 1), ('Jiayou', 1), ('travel', 1), ('leisure', 1), ('hostel', 1), ('Grafs', 1), ('Adler', 1), ('Don', 1), ('Rodrigo', 1),

In [35]:
# transform into a data frame
word_count = pd.DataFrame({'word': [item[0] for item in list(word_count)], 
             'count' : [item[1] for item in list (word_count)]})
# name columns
word_count = word_count[['word', 'count']]
# format
word_count = word_count.sort_values('count', ascending = False)


In [36]:
word_count.head(10)

,word,count
7,Hotel,205
14,Inn,51
116,&,26
96,Resort,22
40,The,21
81,Suites,20
56,-,11
72,La,11
91,Beach,11
117,Spa,11


### Removing Stopwords

In [37]:
stopwords = word_count[word_count["count"]>=10]

In [38]:
stopwords

,word,count
7,Hotel,205
14,Inn,51
116,&,26
96,Resort,22
40,The,21
81,Suites,20
56,-,11
72,La,11
91,Beach,11
117,Spa,11


In [39]:
def remove_stopwords(input_string, stopwords_list): #define function that takes out stopwords from a list of strings and joins strings again
    my_string = []
    for word in input_string.split(): 
        if word not in stopwords_list: 
            my_string.append(word)

    return " ".join(my_string)

stopword_list = stopwords['word'].tolist()
accomodation["cut"] = accomodation['name'].apply(lambda row: remove_stopwords(row, stopword_list)) #apply to 


In [40]:
accomodation["cut2"] = accomodation.name.str.split().apply(lambda x: " ".join([y for y in x if y not in list(stopwords["word"])]))
# SECOND METHOD


In [41]:
accomodation

,name,city,...,cut,cut2
0,Pensiunea Sovirag,Sic,...,Pensiunea Sovirag,Pensiunea Sovirag
1,Bio-Vitalhotel Falkenhof,Bad FĂĽssing,...,Bio-Vitalhotel Falkenhof,Bio-Vitalhotel Falkenhof
2,Duvan Hotell,Uppsala,...,Duvan Hotell,Duvan Hotell
3,Wessex Hotel,Street Nr Glastonbury,...,Wessex,Wessex
4,Alexandria Hotel,Thessaloniki,...,Alexandria,Alexandria
...,...,...,...,...,...
495,Casa Grande Boutique Hotel,Ciudad Bolivar,...,Casa Grande Boutique,Casa Grande Boutique
496,CabaĂ±as Arepo - Barrancas de Lolita,Areponapuchi,...,CabaĂ±as Arepo Barrancas de Lolita,CabaĂ±as Arepo Barrancas de Lolita
497,Lanting Inn,Dunhuang,...,Lanting,Lanting
498,Hotel Belveder,Pag,...,Belveder,Belveder


## Some Regex


In [42]:
data = [["mcdonalds split central22223 london"], ["pizza hut split downtown 3333new york"], ["dominos split new 2222jersey"], [ "mac donald split333 berlin"] ]

restaurants = pd.DataFrame(data, columns = ['restaurant_name'])

restaurants

,restaurant_name
0,mcdonalds split central22223 london
1,pizza hut split downtown 3333new york
2,dominos split new 2222jersey
3,mac donald split333 berlin


In [117]:
restaurants["after"]=restaurants["restaurant_name"].str.replace('.*split',"") #match everything after
restaurants["before"] = restaurants["restaurant_name"].str.replace('split.*',"") #match everything before
restaurants["letters"] = restaurants["restaurant_name"].str.replace('[a-zA-Z]',"") #match letters only 
restaurants["numbers"] = restaurants["restaurant_name"].str.replace('[0-9]',"") #match numbers only 

In [118]:
restaurants

,restaurant_name,before,after,letters,numbers
0,mcdonalds split central22223 london,mcdonalds,central22223 london,22223,mcdonalds split central london
1,pizza hut split downtown 3333new york,pizza hut,downtown 3333new york,3333,pizza hut split downtown new york
2,dominos split new 2222jersey,dominos,new 2222jersey,2222,dominos split new jersey
3,mac donald split333 berlin,mac donald,333 berlin,333,mac donald split berlin
